In [91]:
import numpy as np
from ortools.linear_solver import pywraplp
import random as rd

FILENAME = "/home/ubuntu/Downloads/data_samples_mini_projects/miniproject-20/data.txt"

## Get data from user's input

In [92]:
def get_data(filename):
    """
    Process user's input and generate data.
    
    N    number of articles
    M    number of scientists
    K    minimal number of scientists working on each article
    L    matrix representation of data. L[i, j] = 1 means scientist j can take on article i
    """
    infile = open(filename, 'r')
    
    N, M, K = [int(i) for i in infile.readline().split()]
    
    A = []
    L = np.zeros((N,M), dtype = 'int')
    for i in range(N):
        A.append(sorted([int(k) for k in infile.readline().split()[1:]]))
        for j in A[i]:
            L[i, j - 1] = 1
    return N, M, K, A, L

## Solve the problem with OR-tools

In [93]:
def ortools():
    """Minimize the maximal number of articles that a scientist can take on and print the assignment."""
    
    solver = pywraplp.Solver.CreateSolver('SCIP')
    INF = solver.infinity()

    X = np.array([[None]*M for _ in range(N)])
    Y = solver.IntVar(-INF, INF, 'Y')

    for i in range(N):
        for j in range(M):
            X[i, j] = solver.IntVar(0, int(L[i, j]), 'X[{}, {}]'.format(i, j))

    for i in range(N):
        solver.Add(K == sum(L[i, j]*X[i, j] for j in range(M)))

    for j in range(M):
        solver.Add(Y >= sum(L[i, j]*X[i, j] for i in range(N)))

    solver.Minimize(Y)
    status = solver.Solve()
    

    if status == pywraplp.Solver.OPTIMAL:
        for i in range(N):
            for j in range(M):
                if X[i, j].solution_value() == 1:
                    print(j + 1, end=' ')
            print()
    else:
        print("No feasible solution")

## Solve the problem with Heuristics

In [94]:
def heuristics():
    L = [[] for i in range(N)]
    for i in range(N):
        for j in range(M):
            if l[i,j] == 1:
                L[i].append(j+1)


    current = [[a + 1, 0] for a in range(M)]
    X= [[]for i in range(N)]
    for i in range(N):
        list = []
        d = 0
        while len(list) < K:
            for j in L[i]:
                if current[j-1][1] == min([current[a-1][1] for a in L[i]]) + d:
                    list.append(j)
            d += 1
        result = rd.sample(list,K)
        X[i] = result
        for j in result:
            current[j - 1][1] += 1

    for i in range(N):
        X[i].sort()
    for i in range(N):
        for j in range(K):
            print(X[i][j], end=' ')
        print()

## Solve the problem with Backtracking

In [95]:


def backtracking():
    
    
    def Try(i, t):
        for sci_index_in_A in range(sci_index_in_A_taking[i][t - 1] + 1, len(A[i]) - K + t + 1):
            print("Artical:", i + 1, "Index:", sci_index_in_A + 1, "of range", sci_index_in_A_taking[i][t - 1] + 2, "to", len(A[i]) - K + t + 1)
            sci_index_in_A_taking[i][t] = sci_index_in_A
            queue.append((i, t, sci_index_in_A_taking[i]))
            print(queue)
            #print(A[i], sci_index_in_A)
            number_taked[A[i][sci_index_in_A] - 1] += 1
            #print(sci_index_in_A_taking)
            if i == N - 1 and t == K - 1:
                print("Updating")
                Update()
            else:
                if t < K - 1:
                    Try(i, t + 1)
                else:
                    Try(i + 1, 0)
            number_taked[A[i][sci_index_in_A] - 1] -= 1
            queue.pop()
                
    def Update():
        global obj_val, opt_sol
        if max(number_taked) < obj_val:
            obj_val = max(number_taked)
            opt_sol = sci_index_in_A_taking[:][:]
        #print(number_taked, [[A[i][j] for j in opt_sol[i]] for i in range(N)])
        

            
    Try(0, 0)



## Main function

In [96]:
N, M, K, A, L = get_data(FILENAME)

print(L)

obj_val, opt_sol = float('inf'), None
sci_index_in_A_taking = [[-1]*K for _ in range(N)]
number_taked = [0]*M
queue = []

backtracking()
final_sol = [[A[i][j] for j in opt_sol[i]] for i in range(N)]
print(obj_val, final_sol, end='\n')
        


[[1 1 0 1 0 1 1]
 [1 1 1 1 1 0 1]
 [1 1 0 0 0 1 1]
 [1 1 0 1 1 0 1]
 [1 1 1 0 0 1 1]
 [1 0 1 1 1 1 0]
 [1 1 0 0 0 1 0]
 [0 1 0 1 1 0 1]
 [1 0 1 1 1 0 0]
 [1 1 1 1 1 0 0]]
Artical: 1 Index: 1 of range 1 to 3
[(0, 0, [0, -1, -1])]
Artical: 1 Index: 2 of range 2 to 4
[(0, 0, [0, 1, -1]), (0, 1, [0, 1, -1])]
Artical: 1 Index: 3 of range 3 to 5
[(0, 0, [0, 1, 2]), (0, 1, [0, 1, 2]), (0, 2, [0, 1, 2])]
Artical: 2 Index: 1 of range 1 to 4
[(0, 0, [0, 1, 2]), (0, 1, [0, 1, 2]), (0, 2, [0, 1, 2]), (1, 0, [0, -1, -1])]
Artical: 2 Index: 2 of range 2 to 5
[(0, 0, [0, 1, 2]), (0, 1, [0, 1, 2]), (0, 2, [0, 1, 2]), (1, 0, [0, 1, -1]), (1, 1, [0, 1, -1])]
Artical: 2 Index: 3 of range 3 to 6
[(0, 0, [0, 1, 2]), (0, 1, [0, 1, 2]), (0, 2, [0, 1, 2]), (1, 0, [0, 1, 2]), (1, 1, [0, 1, 2]), (1, 2, [0, 1, 2])]
Artical: 3 Index: 1 of range 1 to 2
[(0, 0, [0, 1, 2]), (0, 1, [0, 1, 2]), (0, 2, [0, 1, 2]), (1, 0, [0, 1, 2]), (1, 1, [0, 1, 2]), (1, 2, [0, 1, 2]), (2, 0, [0, -1, -1])]
Artical: 3 Index: 2 of range